# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235314313664                   -0.50    7.0
  2   -7.250106077970       -1.83       -1.39    1.0
  3   -7.250778594233       -3.17       -1.75    4.0
  4   -7.250915885485       -3.86       -1.85    2.0
  5   -7.251331684373       -3.38       -2.69    2.0
  6   -7.251338277330       -5.18       -3.21    2.0
  7   -7.251338779684       -6.30       -3.88    2.0
  8   -7.251338794830       -7.82       -4.17    3.0
  9   -7.251338798525       -8.43       -4.73    2.0
 10   -7.251338798680       -9.81       -5.32    2.0
 11   -7.251338798701      -10.66       -5.82    3.0
 12   -7.251338798704      -11.53       -6.55    2.0
 13   -7.251338798705      -13.12       -6.95    3.0
 14   -7.251338798705      -13.97       -7.42    2.0
 15   -7.251338798705      -14.05       -8.10    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08851872482341214
[ Info: Arnoldi iteration step 2: normres = 0.4524401464036587
[ Info: Arnoldi iteration step 3: normres = 0.9171121180848835
[ Info: Arnoldi iteration step 4: normres = 0.23502035278006775
[ Info: Arnoldi iteration step 5: normres = 0.6175782816699701
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.70e-02, 6.93e-02, 5.74e-01, 2.14e-01, 2.46e-03)
[ Info: Arnoldi iteration step 6: normres = 0.23542500804335315
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (7.23e-03, 1.56e-01, 1.04e-01, 1.01e-01, 5.02e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09183570814984225
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.23e-04, 1.31e-02, 6.83e-03, 6.51e-02, 5.53e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12839471373667652
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.79e